In [ ]:
%matplotlib inline

import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import gzip
import random
import numpy as np

from scipy import stats
from operator import itemgetter
import seaborn as sns

import re

In [ ]:
users = []
with gzip.open('province_gender_available_metadata_added-220210.txt.gz', 'rb') as f:
    for l in f:
        user_object = json.loads(l)
        users.append(user_object)

In [ ]:
len(users)

In [ ]:
users[1]

In [ ]:
types=[]
for user in users:
    for tweet in user["tweets"]:
        if tweet["type"] not in types:
            types.append(tweet["type"])
        else:
            continue

print(types)

In [ ]:
dict_dataframe= dict()
created_at_user_list=[] #the time that tweet is sent from that account (either original, retweeted or fav)
user_id_list=[] #the id of the user
user_screen_name_list=[] #the screen name of the user
tweet_id_list=[] #the id of the tweet
text_list=[] #the tweet
user_mentions_id_list=[] #the user that is the tweet is taken from

in_reply_to_status_id_list=[] #the tweet id that that tweet is replying to
in_reply_to_user_id_list=[] #the user id that that tweet is replying to

retweeted_id_list=[] #the user id that that tweet is retweeted

fav_user_id_list=[] #the user id that that tweet is faved.


followers_count_list=[]

In [ ]:
for user in users:
    for tweet in user["tweets"]:
        if tweet["type"]=="retweet":
            created_at_user_list.append(user["created_at"])
            user_id_list.append(user["id_str"])
            user_screen_name_list.append(user["screen_name"])
            followers_count_list.append(user["followers_count"])
            text_list.append(tweet["ref_twt_txt"])
            tweet_id_list.append(tweet["ref_twt_id_str"])
            retweeted_id_list.append(tweet["ref_usr_id_str"])
            in_reply_to_status_id_list.append(np.nan)
            in_reply_to_user_id_list.append(np.nan)
            user_mentions_id_list.append(np.nan)
            fav_user_id_list.append(np.nan)
            
        if tweet["type"]=="fav":
            created_at_user_list.append(user["created_at"])
            user_id_list.append(user["id_str"])
            user_screen_name_list.append(user["screen_name"])
            followers_count_list.append(user["followers_count"])
            text_list.append(tweet["ref_twt_txt"])
            tweet_id_list.append(tweet["ref_twt_id_str"])
            retweeted_id_list.append(np.nan)
            in_reply_to_status_id_list.append(np.nan)
            in_reply_to_user_id_list.append(np.nan)
            user_mentions_id_list.append(np.nan)
            fav_user_id_list.append(tweet["ref_usr_id_str"])
            
        if tweet["type"]=="quote":
            created_at_user_list.append(user["created_at"])
            user_id_list.append(user["id_str"])
            user_screen_name_list.append(user["screen_name"])
            followers_count_list.append(user["followers_count"])
            text_list.append(tweet["ref_twt_txt"])
            tweet_id_list.append(tweet["twt_id_str"])
            retweeted_id_list.append(np.nan)
            in_reply_to_status_id_list.append(np.nan)
            in_reply_to_user_id_list.append(np.nan)
            user_mentions_id_list.append(tweet["ref_usr_id_str"])
            fav_user_id_list.append(np.nan)
            
        if tweet["type"]=="reply":
            created_at_user_list.append(user["created_at"])
            user_id_list.append(user["id_str"])
            user_screen_name_list.append(user["screen_name"])
            followers_count_list.append(user["followers_count"])
            text_list.append(tweet["ref_twt_txt"])
            tweet_id_list.append(tweet["twt_id_str"])
            retweeted_id_list.append(np.nan)
            in_reply_to_status_id_list.append(tweet["ref_usr_id_str"])
            in_reply_to_user_id_list.append(tweet["ref_twt_id_str"])
            user_mentions_id_list.append(tweet["ref_usr_id_str"])
            fav_user_id_list.append(np.nan)
            
        if tweet["type"]=="original":
            created_at_user_list.append(user["created_at"])
            user_id_list.append(user["id_str"])
            user_screen_name_list.append(user["screen_name"])
            followers_count_list.append(user["followers_count"])
            text_list.append(tweet["twt_txt"])
            tweet_id_list.append(tweet["twt_id_str"])
            retweeted_id_list.append(np.nan)
            in_reply_to_status_id_list.append(np.nan)
            in_reply_to_user_id_list.append(np.nan)
            user_mentions_id_list.append(np.nan)
            fav_user_id_list.append(np.nan)
            

In [ ]:
print("user_id_list",len(user_id_list))
print("created_at_user_list",len(created_at_user_list))
print("user_screen_name_list",len(user_screen_name_list))
print("tweet_id_list",len(tweet_id_list))
print("text_list",len(text_list))
print("user_mentions_id_list",len(user_mentions_id_list))
print("in_reply_to_status_id_list",len(in_reply_to_status_id_list))
print("in_reply_to_user_id_list",len(in_reply_to_user_id_list))
print("retweeted_id_list",len(retweeted_id_list))
print("fav_user_id_list",len(fav_user_id_list))


In [ ]:
dict_dataframe["created_at_user"] = created_at_user_list
dict_dataframe["user_id"] = user_id_list
dict_dataframe["screen_name"] = user_screen_name_list
dict_dataframe["text"] = text_list
dict_dataframe["tweet_id"] = tweet_id_list
dict_dataframe["in_reply_to_status_id"] = in_reply_to_status_id_list
dict_dataframe["in_reply_to_user_id"] = in_reply_to_user_id_list
dict_dataframe["retweeted_id"] = retweeted_id_list
dict_dataframe["fav_user_id"] = fav_user_id_list
dict_dataframe["user_mentions_id"] = user_mentions_id_list
dict_dataframe["followers_count"] = followers_count_list


In [ ]:
df = pd.DataFrame(dict_dataframe)
df = df.where((pd.notnull(df)), None)

In [ ]:
df

In [ ]:
def get_interactions(row):
    user = row["user_id"], row["screen_name"]
    if user[0] is None:
        return (None, None), []

    interactions = set()
    interactions.add((row["in_reply_to_user_id"]))
    interactions.add((row["retweeted_id"]))
    interactions.add((row["fav_user_id"]))
    
    interactions.discard((row["user_id"], row["screen_name"]))
    interactions.discard((None, None))
    
    return user, interactions


In [ ]:
graph = nx.Graph()

In [ ]:
for index, tweet in df.iterrows():
    user, interactions = get_interactions(tweet)
    user_id, user_name = user
    tweet_id = tweet["tweet_id"]
    for interaction in interactions:
        int_id = interaction
        graph.add_edge(user_id, int_id, tweet_id=tweet_id)

In [ ]:
print(f"There are {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges present in the Graph")

In [ ]:
degrees = [val for (node, val) in graph.degree()]

In [ ]:
print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")

In [ ]:
print(f"The average degree of the nodes in the Graph is {np.mean(degrees):.1f}")  
print(f"The most frequent degree of the nodes found in the Graph is {stats.mode(degrees)[0][0]}")

In [ ]:
if nx.is_connected(graph):
    print("The graph is connected")
else:
    print("The graph is not connected")

In [ ]:
#if the graph is not connected, then we can check the connected components in the inconnected craph. 
print(f"There are {nx.number_connected_components(graph)} connected components in the Graph")

In [ ]:
largest_subgraph = max((graph.subgraph(c) for c in nx.connected_components(graph)), key=len)
print(f"There are {largest_subgraph.number_of_nodes()} nodes and {largest_subgraph.number_of_edges()} \
edges present in the largest component of the Graph")

In [ ]:
if nx.is_connected(largest_subgraph):
    print("The graph is connected")
else:
    print("The graph is not connected")

In [ ]:
print(f"The average clustering coefficient is {nx.average_clustering(largest_subgraph)} in the largest subgraph")
print(f"The transitivity of the largest subgraph is {nx.transitivity(largest_subgraph)}")

In [ ]:
graph_centrality = nx.degree_centrality(largest_subgraph)

In [ ]:
max_de = max(graph_centrality.items(), key=itemgetter(1))

In [ ]:
graph_closeness = nx.closeness_centrality(largest_subgraph)

In [ ]:
max_clo = max(graph_closeness.items(), key=itemgetter(1))

In [ ]:
graph_betweenness = nx.betweenness_centrality(largest_subgraph, normalized=True, endpoints=False)

In [ ]:
max_bet = max(graph_betweenness.items(), key=itemgetter(1))

In [ ]:
print(f"the node with id {max_de[0]} has a degree centrality of {max_de[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_clo[0]} has a closeness centrality of {max_clo[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_bet[0]} has a betweenness centrality of {max_bet[1]:.2f} which is the maximum of the Graph")

In [ ]:
node_and_degree = largest_subgraph.degree()
colors_central_nodes = ['orange', 'red']

In [ ]:
pos = nx.spring_layout(largest_subgraph, k=0.05)

In [ ]:
plt.figure(figsize = (20,20))
nx.draw(largest_subgraph, pos=pos, node_color=range(1404), cmap=plt.cm.PiYG, edge_color="black", linewidths=0.3, node_size=60, alpha=0.6, with_labels=False)
nx.draw_networkx_nodes(largest_subgraph, pos=pos, nodelist=central_nodes, node_size=300, node_color=colors_central_nodes)
#plt.savefig('graphfinal.png')
plt.show()